## MLC HW 4

### Question 1 (40 points)

In this question, you will model traffic counts in Pittsburgh using Gaussian process (GP) regression.  The included dataset, "PittsburghTrafficCounts.csv", represents the average daily traffic counts computed by traffic sensors at over 1,100 locations in Allegheny County, PA.  The data was collected from years 2012-2014 and compiled by Carnegie Mellon University’s Traffic21 Institute; we have the longitude, latitude, and average daily count for each sensor.  

Given this dataset, your goal is to learn a model of traffic count as a function of spatial location.  To do so, fit a Gaussian Process regression model to the observed data.  While you can decide on the precise kernel specification, you should try to achieve a good model fit, as quantified by a log marginal likelihood value greater than (i.e., less negative than) -1400.  Here are some hints for getting a good model fit:

* We recommend that you take the logarithm of the traffic counts, and then subtract the mean of this vector, before fitting the model.

* Since the data is noisy, don't forget to include a noise term (WhiteKernel) in your model.

* When fitting a GP with RBF kernel on multidimensional data, you can learn a separate length scale for each dimension, e.g., length_scale=(length_scale_x, length_scale_y). 

Your Python code should provide the following five outputs:

1) The kernel after parameter optimization and fitting to the observed data. (10 pts)

2) The log marginal likelihood of the training data. (5 pts)

3) Show a 2-D plot of the model's predictions over a mesh grid of longitude/latitude (with color corresponding to the model's predictions) and overlay a 2-D scatter plot of sensor locations (with color corresponding to the observed values). (10 pts)

4) What percentage of sensors have average traffic counts more than two standard deviations higher or lower than the model predicts given their spatial location? (5 pts)

5) Show a 2-D scatter plot of the sensor locations, with three colors corresponding to observed values a) more than two standard deviations higher than predicted, b) more then two standard deviations lower than predicted, and c) within two standard deviations of the predicted values. (10 pts)


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel,WhiteKernel
from sklearn.model_selection import train_test_split
Data1=pd.read_csv("PittsburghTrafficCounts.csv")
Data1.head()

,Longitude,Latitude,AvgDailyTrafficCount
0,-80.278366,40.468606,84.0
1,-80.162117,40.384598,95.0
2,-80.221205,40.366778,97.0
3,-80.142455,40.622084,111.0
4,-80.131975,40.544915,125.0


In [2]:
y = np.array(np.log(Data1['AvgDailyTrafficCount']) - np.log(Data1['AvgDailyTrafficCount']).mean())
X = np.array(Data1[['Longitude','Latitude']])

In [3]:
X_train = X[:800]
y_train = y[:800]

In [6]:
kernel = 1.* RBF(length_scale=0.0567, length_scale_bounds=(1e-3, 1e3)) \
    + WhiteKernel(noise_level=0.599, noise_level_bounds=(1e-10, 1e10))

gpc_rbf = GaussianProcessRegressor(kernel=kernel).fit(X_train, y_train)

print (gpc_rbf.log_marginal_likelihood_value_)
print (gpc_rbf.kernel_)


-987.2049501517168
0.712**2 * RBF(length_scale=0.0567) + WhiteKernel(noise_level=0.599)


In [9]:
h = 0.01
lat_min, lat_max = X[:,1].min() - 0.01, X[:,1].max() + 0.01
lng_min, lng_max = X[:,0].min() - 0.01, X[:,0].max() + 0.01

lat,lng = np.meshgrid(np.arange(lat_min, lat_max, h),
                     np.arange(lng_min, lng_max, h))

x = [i for i in zip(lng.flat,lat.flat)]
z = gpc_rbf.predict(x)
z = z.reshape(len(np.arange(lng_min, lng_max, h)),len(np.arange(lat_min, lat_max, h)))

In [30]:
z.min()

-2.088531031123593

In [31]:
z.max()

0.4361354357893594

No handles with labels found to put in legend.


In [55]:
plt.figure(figsize=(6,6))
cs = plt.contourf(lat, lng, z, cmap='coolwarm_r',legend=True,levels=np.arange(-2.1, 0.6, 0.1).tolist())
plt.scatter(X[:,1], X[:,0], c=y, cmap='coolwarm_r',marker='.')
proxy = [plt.Rectangle((0,0),1,1,fc = pc.get_facecolor()[0]) for pc in cs.collections]

plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.legend()

proxy = [plt.Rectangle((0,0),1,1,fc = pc.get_facecolor()[0]) 
for pc in cs.collections]

plt.legend(proxy, np.arange(-2.1, 0.6, 0.1).tolist(),bbox_to_anchor=(1.6, 0.5), loc='right', ncol=1)
plt.show()

### Question 2: Cluster-based anomaly detection (10 points)

Given an unlabeled dataset with two real-valued attributes, we perform cluster-based anomaly detection by running k-means, choosing the number of clusters k automatically using the Schwarz criterion.  Four clusters are formed:

A: 100 points, center (0, 0), standard deviation 0.1

B: 150 points, center (35, 5), standard deviation 5

C: 2 points, center (15, 20), standard deviation 1

D: 200 points, center (10, 10), standard deviation 1

Given the four points below, which of these points are, and are not, likely to be anomalies?  Choose “Anomaly” or “Not Anomaly”, and provide a brief explanation, for each point. 
(Hint: your answers should take into account the size and standard deviation of each cluster as well as the distances to cluster centers.)

(1, 0)		Anomaly / Not Anomaly


(35, 2)		Anomaly / Not Anomaly


(15, 19)	Anomaly / Not Anomaly


(10, 11)	Anomaly / Not Anomaly


#### Your solution here
- (1,0) Anomaly. As this point is 10 std away from the center point, and the amount of 100 points is enough the support the conclude of a common distribution which this point is far away from.
- (35,2) Not Anomaly. The distance between (35,2) and (35,5) is less than 1 std, and the amount of 150 points is enough the support the conclude of a common distribution which this point is far away from.
- (15,19) Unable to decide. The distance between this point and the center is 1 std, but the amount of points in the cluster is too less to conclude a standard distribution.
- （10，11）Anomaly. The distance between (10,11) and (10,11) is l std of the data set, and the amount of 200 points is enough the support the conclude of a common distribution.

### Question 3: Anomaly detection (50 points)

For this question, use the "County Health Indicators" dataset provided to identify the most anomalous counties. Please list the top 5 most anomalous counties computed using each of the following models.  (We recommend that, as a pre-processing step, you drop na values, and make sure all numeric values are treated as floats not strings.)

Part 1: Learn a Bayesian network structure using only the six features ["'\% Smokers'","'\% Obese'","'Violent Crime Rate'","'80/20 Income Ratio'","'\% Children in Poverty'","'Average Daily PM2.5'"].  Use pd.cut() to discretize each feature into 5 categories: 0,1,2,3,4. 

(a) Use HillClimbSearch and BicScore to learn the Bayesian network structure (5 pts) 

(b) Which 5 counties have the lowest (most negative) log-likelihood values? Please show a ranked list of the top counties' names and log-likelihood values. (10 pts)

Part 2: Cluster based anomaly detection.  Use all numeric features for this part, and do not discretize. 

(a) Clustering with k-means. Please use k=3 clusters. Compute each record's distance to the nearest cluster center and report the five counties which have the longest distances. (10 pts)

(b) Cluster with Gaussian Mixture. Please repeat (2)a but use log-likelihood for each record (rather than distance) as the measure of anomalousness. (10 pts)

Part 3: Choose one more anomaly detection model you prefer and report the top 5 most anomalous counties by the model you chose.  (10 pts)

Part 4: Compare and contrast the results from the different models.  Were there some counties that were found to be anomalous in some models and not in others?  Please provide some intuitions on why each county was found to be anomalous. (5 pts)

In [ ]:
Data2=pd.read_csv("2016CountyHealthIndicators.csv")
Data2.head()

In [ ]:
# your code here